### Spot Instances

**Execute a script**
https://peteris.rocks/blog/script-to-launch-amazon-ec2-spot-instances/

**Open a Notebook**
https://chrisalbon.com/aws/basics/run_project_jupyter_on_amazon_ec2/

*There are programmatic ways to retrieve both pricing and the launch wizard but these can take time so for a quick reference refer to the solutions below.*

**Prices** : https://aws.amazon.com/ec2/spot/pricing/
<br>**Images** : go to the launch-wizard to get the image IDs or use the boto3 client.describe_images() with Owners of Filters parameters to reduce wait time and find what you need.

**Storage**: storage options for spot instances include S3, EBS and EFS. Spot instances can be optimized for EBS. For a breakdown of storage options read [this](https://dzone.com/articles/confused-by-aws-storage-options-s3-ebs-amp-efs-explained) article. 

In [2]:
try:
    import awscli
except:
    import pip
    pip.main(['install', 'awscli'])    
    import awscli

In [15]:
import boto3 

client = boto3.client('ec2')

In [ ]:
'''
AWS datasync is a process specialized in transfering data. There are 4 components to a datasync: 
- Agent: A virtual machine used to read data from or write data.
- Location: Any source or destination location used in the data transfer (for example, Amazon S3 or Amazon EFS).
- Task: A task includes two locations, and also the configuration of how to transfer the data. A task is the complete definition of a data transfer.
- Task execution: An individual run of a task, which includes options such as start time, end time, bytes written, and status.

There are 5 tasks to the datasync: 
- Step 1: Create an Agent
- Step 2: Create Locations
- Step 3: Create a Task
- Step 4: Start a Task Execution
- Step 5: Monitor Your Task Execution
'''

# STEP 1: Create an agent 
# The agent can be an ec2 instance with a special datasync image AMI provided by AWS. 
# Each datasync image is specific to its region. 
# For a list of regional-images see: https://docs.aws.amazon.com/datasync/latest/userguide/deploy-agents.html#ec2-deploy-agent
# Launching an ec2 instance also requires picking an instance type
# For recommended instance types see: https://docs.aws.amazon.com/datasync/latest/userguide/requirements.html#ec2-instance-types

# AWS Does not recommend transfering directly from a local source to EFS so first we'll upload our data to S3 which is pretty cheap. 


In [22]:
import spot_connect as sc 
from importlib import reload
sc = reload(sc)

### EFS Storage

**Boto3 documentation** https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/efs.html#EFS.Client.create_file_system

In [3]:

# For testing purposes: get a currently active and running instance, the type for which we would recieve 
ec2 = boto3.client('ec2', region_name='us-west-2')
instance = ec2.describe_instances()['Reservations'][0]['Instances'][0]
security_group = ec2.describe_security_groups(Filters=[{'Name':'group-name','Values':['SG-test']}])['SecurityGroups'][0]

In [4]:
import boto3 
from netaddr import IPNetwork

client = boto3.client('efs', region_name='us-west-2')

file_systems = client.describe_file_systems()['FileSystems']                  # Check if there are any existing filesystems 
if len(file_systems)==0:                                                      # If not 
    print('Creating EFS file system...')
    response = client.create_file_system(
        CreationToken='test_efs',
        PerformanceMode='generalPurpose',
    )
    initiated=False 
    print('Initializing...')
    while not initiated: 
        try: 
            file_system = client.describe_file_systems()['FileSystems'][0]
            file_system_id = file_system['FileSystemId']
        except: 
            print('.')
else: 
    file_system = file_systems[0]
    file_system_id = file_system['FileSystemId']
    
    
    


In [6]:
client = boto3.client('efs', region_name='us-west-2')

In [7]:
help(client.describe_file_systems)

Help on method describe_file_systems in module botocore.client:

describe_file_systems(*args, **kwargs) method of botocore.client.EFS instance
    Returns the description of a specific Amazon EFS file system if either the file system ``CreationToken`` or the ``FileSystemId`` is provided. Otherwise, it returns descriptions of all file systems owned by the caller's AWS account in the AWS Region of the endpoint that you're calling.
    
     
    
    When retrieving all file system descriptions, you can optionally specify the ``MaxItems`` parameter to limit the number of descriptions in a response. If more file system descriptions remain, Amazon EFS returns a ``NextMarker`` , an opaque token, in the response. In this case, you should send a subsequent request with the ``Marker`` request parameter set to the value of ``NextMarker`` . 
    
     
    
    To retrieve a list of your file system descriptions, this operation is used in an iterative process, where ``DescribeFileSystems`` is ca

In [8]:
help(client.describe_mount_targets)

Help on method describe_mount_targets in module botocore.client:

describe_mount_targets(*args, **kwargs) method of botocore.client.EFS instance
    Returns the descriptions of all the current mount targets, or a specific mount target, for a file system. When requesting all of the current mount targets, the order of mount targets returned in the response is unspecified.
    
     
    
    This operation requires permissions for the ``elasticfilesystem:DescribeMountTargets`` action, on either the file system ID that you specify in ``FileSystemId`` , or on the file system of the mount target that you specify in ``MountTargetId`` .
    
    
    
    See also: `AWS API Documentation <https://docs.aws.amazon.com/goto/WebAPI/elasticfilesystem-2015-02-01/DescribeMountTargets>`_
    
    
    **Request Syntax** 
    ::
    
      response = client.describe_mount_targets(
          MaxItems=123,
          Marker='string',
          FileSystemId='string',
          MountTargetId='string'
     

In [124]:
subnet_id = instance['SubnetId']

ec2 = boto3.resource('ec2')
subnet = ec2.Subnet(subnet_id)
net = IPNetwork(subnet.cidr_block)
ips = [str(x) for x in list(net[4:-1])]

In [122]:
response = client.create_mount_target(
    FileSystemId=file_system_id,
    SubnetId=subnet_id,
    IpAddress=ips[0],
    SecurityGroups=[
        security_group_id,
    ]
)

In [126]:
response

{'FileSystemId': 'fs-be035a15',
 'IpAddress': '172.31.0.4',
 'LifeCycleState': 'creating',
 'MountTargetId': 'fsmt-f429935c',
 'NetworkInterfaceId': 'eni-0158d2ccffea8297d',
 'OwnerId': '006326790825',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '214',
   'content-type': 'application/json',
   'date': 'Thu, 05 Sep 2019 19:42:33 GMT',
   'x-amzn-requestid': 'ea62593f-252e-47a7-8e02-6b0832f62f51'},
  'HTTPStatusCode': 200,
  'RequestId': 'ea62593f-252e-47a7-8e02-6b0832f62f51',
  'RetryAttempts': 0},
 'SubnetId': 'subnet-79f21324'}

In [146]:
instance_dns = instance['PublicDnsName']
instance_dns

'ec2-34-219-28-255.us-west-2.compute.amazonaws.com'

In [13]:
import os 
os.getcwd() 

'C:\\Users\\carlo\\Dropbox\\Projects\\Spot-Instance-AWS'

In [17]:
with open('dns_info.sh','w') as f:                                         # https://docs.aws.amazon.com/efs/latest/ug/wt1-test.html
    f.write('sudo yum -y install nfs-utils'+'\n')
    f.write('mkdir ~/efs-mount-point'+'\n')
    f.write('sudo mount -t nfs -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2,noresvport '+filesystem_dns+':/   ~/efs-mount-point '+'\n')
    f.write('cd ~/efs-mount-point'+'\n')
    f.write('sudo chmod go+rw .'+'\n')
    f.close() 

In [16]:
with open('dns_info.txt','r') as f: 
    text = f.read() 
    instance_dns = text.split('\n')[0]
    filesystem_dns = text.split('\n')[1]
    
print(instance_dns, filesystem_dns)

ec2-34-219-28-255.us-west-2.compute.amazonaws.com fs-be035a15.efs.us-west-2.amazonaws.com


In [147]:
filesystem_dns = file_system_id+'.efs.'+'us-west-2'+'.amazonaws.com'
filesystem_dns

'fs-be035a15.efs.us-west-2.amazonaws.com'

In [9]:
help(client.describe_file_systems)

Help on method describe_file_systems in module botocore.client:

describe_file_systems(*args, **kwargs) method of botocore.client.EFS instance
    Returns the description of a specific Amazon EFS file system if either the file system ``CreationToken`` or the ``FileSystemId`` is provided. Otherwise, it returns descriptions of all file systems owned by the caller's AWS account in the AWS Region of the endpoint that you're calling.
    
     
    
    When retrieving all file system descriptions, you can optionally specify the ``MaxItems`` parameter to limit the number of descriptions in a response. If more file system descriptions remain, Amazon EFS returns a ``NextMarker`` , an opaque token, in the response. In this case, you should send a subsequent request with the ``Marker`` request parameter set to the value of ``NextMarker`` . 
    
     
    
    To retrieve a list of your file system descriptions, this operation is used in an iterative process, where ``DescribeFileSystems`` is ca

In [22]:
region = 'us-west-2'
client = boto3.client('efs', region_name=region)

In [28]:
client.describe_file_systems(CreationToken='test')['FileSystems'][0]

{'CreationTime': datetime.datetime(2019, 9, 6, 18, 59, 34, tzinfo=tzlocal()),
 'CreationToken': 'test',
 'Encrypted': False,
 'FileSystemId': 'fs-edbee646',
 'LifeCycleState': 'available',
 'NumberOfMountTargets': 0,
 'OwnerId': '006326790825',
 'PerformanceMode': 'generalPurpose',
 'SizeInBytes': {'Value': 6144},
 'ThroughputMode': 'bursting'}

In [10]:
ecs = boto3.client('ec2', region_name='us-west-2')